# Memgraph as Graph Memory

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Install Memgraph

To utilize Memgraph as Graph Memory, run it with Docker:

```bash
docker run -p 7687:7687 memgraph/memgraph-mage:latest --schema-info-enabled=True
```

The `--schema-info-enabled` flag is set to `True` for more performant schema
generation.

Additional information can be found on [Memgraph documentation](https://memgraph.com/docs). 

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [ ]:
from mem0 import Memory

import os

os.environ["OPENAI_API_KEY"] = ""

Set up configuration to use the embedder model and Memgraph as a graph store:

In [15]:
config = {
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-large", "embedding_dims": 1536},
    },
    "graph_store": {
        "provider": "memgraph",
        "config": {
            "url": "bolt://localhost:7687",
            "username": "memgraph",
            "password": "mem0graph",
        },
    },
}

## Graph Memory initializiation 

Initialize Memgraph as a Graph Memory store: 

In [16]:
m = Memory.from_config(config_dict=config)

/Users/katelatte/repos/forks/mem0/.venv/lib/python3.13/site-packages/neo4j/_sync/driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


## Store memories 

Create memories:

In [17]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

Store memories in Memgraph:

In [18]:
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})

![](./alice-memories.png)

## Search memories

In [19]:
for result in m.search("what does alice love?", user_id="alice")["results"]:
    print(result["memory"], result["score"])

Loves sci-fi movies 0.31536642873408993
Planning to watch a movie tonight 0.09684523796547778
Not a big fan of thriller movies 0.09468540071789475
